In [2]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
from fanc_seg_utils.database import Neuron_database
import numpy as np
from matplotlib import pyplot as plt
import pymaid
from scipy.spatial import distance 
from concurrent import futures
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import norm
import pandas as pd
import marshmallow as mm
from cloudvolume import CloudVolume
from fanc_seg_utils import neuroglancer_utilities
import collections
import six
import json
import meshparty
from meshparty import trimesh_io, trimesh_vtk
from annotationframeworkclient import FrameworkClient




# Transfering data to annotation framework
### 1. Get list of spatial points from neuron databases
### 2. Convert them into an annotation layer
### 3. Get the JSON state and save it.
### 4. Add annotations to a new table. 

#### Get database

In [5]:
database_path = '/Users/brandon/Documents/Repositories/Python/Neuron_Database/T1_MN.csv'
db = Neuron_database(database_path,segmentation_version='V4')
db.get_database()


Database active


In [7]:
db.soma_annotation_table()

In [1]:

auth_token = 'N1cxaUtVZ0JSRUxyd05WY3dUVXFCRm8tclNuSk9FTEg='
datastack_name = 'vnc_v0' # from https://api.zetta.ai/wclee/info/

client = FrameworkClient(
    datastack_name,
    server_address = "https://api.zetta.ai/wclee",
    auth_token = auth_token
)

print(f"The image source is: {client.info.image_source()}")

The image source is: precomputed://gs://zetta_lee_fly_vnc_001_precomputed/vnc1_full_v3align_2/realigned_v1


In [8]:
example_id = 495722871693421950
example_state = client.state.get_state_json(example_id)

example_state['layers'][0]['name'] = 'renamed'
new_id = client.state.upload_state_json(example_state)

new_state = client.state.get_state_json(new_id)
print(new_state['layers'][0]['name'])


url = client.state.build_neuroglancer_url(state_id=new_id)
print(url)

renamed
https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/wclee/nglstate/api/v1/405126384626320940


In [9]:
new_state

{'concurrentDownloads': 36,
 'gpuMemoryLimit': 3000000000,
 'jsonStateServer': 'https://api.zetta.ai/json/post',
 'layers': [{'blend': 'default',
   'name': 'renamed',
   'shaderControls': {},
   'source': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/vnc1_full_v3align_2/realigned_v1',
   'type': 'image'},
  {'graphOperationMarker': [{'annotations': [], 'tags': []},
    {'annotations': [], 'tags': []}],
   'name': 'vnc1_full_v3align_2_v2',
   'pathFinder': {'color': '#ffff00',
    'pathObject': {'annotationPath': {'annotations': [], 'tags': []},
     'hasPath': False}},
   'segments': ['648518346595237078', '648518346603712214'],
   'skeletonRendering': {'mode2d': 'lines_and_points', 'mode3d': 'lines'},
   'source': 'graphene://https://api.zetta.ai/wclee/segmentation/table/vnc1_full_v3align_2_v2',
   'type': 'segmentation_with_graph'}],
 'layout': {'orthographicProjection': True, 'type': 'xy-3d'},
 'navigation': {'pose': {'position': {'voxelCoordinates': [6639.61279296875,
    